In [ ]:
# Plot per-node Smith Chart + phase waterfalls

outdir = '/lustre/aoc/projects/hera/dstorer/Projects/highDelayFeature/loadAnalysis/smithCharts'

pols = ['YY']
order='snap'
h = cm_hookup.Hookup()
x = h.get_hookup('HH')

for node in nodes:
    print(f'Node {node}')
    for pol in pols:
        print(f'Pol {pol}')
        if pol =='XX':
            pref = 'N'
        else:
            pref = 'E'
        nodes, antDict, inclNodes = utils.generate_nodeDict(uvd,pol)
        ants = nodes[node]['ants']
        if order=='snap':
            ants = utils.sort_antennas(uvd,use_ants=ants)
        elif order=='pam':
            pams = []
            for a in ants:
                key = f'HH{a}:A'
                p = x[key].get_part_from_type('post-amp')[f'{pref}<ground'][3:]
                pams.append(int(p))
            sorted_ants = []
            for i,p in enumerate(sorted(pams)):
                for j,a in enumerate(ants):
                    key = f'HH{a}:A'
                    pam = x[key].get_part_from_type('post-amp')[f'{pref}<ground'][3:]
                    if int(pam) == p:
                        sorted_ants.append(a)
            ants = sorted_ants
        fig = plt.figure(figsize=(50,50))
        gs = fig.add_gridspec(len(ants),len(ants),hspace=0.25,wspace=0.2,left=0.1)
        corr = calc_corr_metric(uvd,uvd_diff,bls=[(a1,a2,pol) for a1 in ants for a2 in ants])
#         print(corr.keys())
        corrmap = plt.get_cmap('plasma')
        for i,a1 in enumerate(ants):
            print(a1)
            for j,a2 in enumerate(ants):
                s1 = antDict[a1]['snapLocs'][0]
                s2 = antDict[a2]['snapLocs'][0]
                sin1 = antDict[a1]['snapInput'][0]
                sin2 = antDict[a2]['snapInput'][0]
                key1 = f'HH{a1}:A'
                p1 = int(x[key1].get_part_from_type('post-amp')[f'{pref}<ground'][3:])
                key2 = f'HH{a2}:A'
                p2 = int(x[key2].get_part_from_type('post-amp')[f'{pref}<ground'][3:])
#                 print(f'({p1},{p2})')
                d = uvd.get_data(a1,a2,pol)
                freqs = uvd.freq_array[0]*1e-6
                c = corr[(a1,a2,pol)]
                if i <= j:
                    davg = np.average(d,axis=0)
                    r = np.abs(davg)
                    theta = np.angle(davg)
                    min10 = np.log10(np.min(r))
                    max10 = np.log10(np.max(r))
                    r = np.log10(r) - min10
                    ax = fig.add_subplot(gs[i,j],polar=True)
                    im = ax.scatter(theta,r,c =freqs, cmap='viridis',s=20)
                    ax.set_rlim(0,np.percentile(r,99.5))
                    ax.set_rticks([])
                    if a2 == ants[-1]:
                        fig.colorbar(im)
                    ax.set_ylabel('Re')
                    ax.set_xlabel('Im')
                    ax.set_xticklabels([])
                else:
                    ax = fig.add_subplot(gs[i,j])
                    im = ax.imshow(np.angle(d),cmap='twilight',vmin=-np.pi,vmax=np.pi,aspect='auto')
                    if a1 == ants[-1]:
                        fig.colorbar(im,orientation='horizontal')
                        ax.set_xlabel('Frequency')
                    else:
                        ax.set_xticks([])
                    ax.set_yticks([])
                    if a2 == ants[0]:
                        ax.set_ylabel('Time')
                if c < 0.3:
                    ax.set_title(f'({a1},{a2}) - (S{s1}-{sin1}, S{s2}-{sin2}) - (P{p1}, P{p2})',backgroundcolor=corrmap(c),color='w')
                else:
                    ax.set_title(f'({a1},{a2}) - (S{s1}-{sin1}, S{s2}-{sin2}) - (P{p1}, P{p2})',backgroundcolor=corrmap(c),color='black')
        fig.suptitle(f'Node {node} - {pol}')
        print(f'Saving {outdir}/Node{node}_{pol}_smithChart_by_{order}.pdf')
        plt.savefig(f'{outdir}/Node{node}_{pol}_smithChart_by_{order}.pdf',dpi=300)
        plt.close()
    #     plt.show()